In [1]:
import openai

import pandas as pd
import tiktoken

from tqdm import tqdm

import sacrebleu
from rouge_score import rouge_scorer

import os
# from dotenv import load_dotenv, find_dotenv
# _ = load_dotenv(find_dotenv()) # read local .env file

# openai.api_key  = os.getenv('OPENAI_API_KEY')
openai.api_key  = ""

In [2]:
client = openai.AsyncOpenAI(
  api_key=openai.api_key,  # this is also the default, it can be omitted
)


async def get_completion(prompt_, model_="gpt-3.5-turbo"):
    messages_ = [{"role": "user", "content": prompt_}]
    response_ = await client.chat.completions.create(
        model=model_, 
        messages=messages_,
        temperature=0.00000001,
    )
    return response_.choices[0].message.content

# Set up prompt

In [3]:
str_term = "(λx.((λy.((λz.z) x)) (λa.a)))"
next_step_term = "(λx.((λy.y) (λa.a)))"

prompt = f"""
Please generate the next step of reduction a lambda term. Provide only term expression.

Lambda term: '''{str_term}'''
"""
response = await get_completion(prompt)
print(f"expected output: {next_step_term}")
print(f"model output: {response}")

expected output: (λx.((λy.y) (λa.a)))
model output: (λx.((λy.y) (λa.a)))


# Load the data

In [6]:
df = pd.read_csv("./data/term_step_LO.csv", delimiter=",")
df.head()

,term,term_next_LO
0,((λx.(λy.((y y) (λz.z)))) (λa.(((λb.b) (a (λc....,(λx.((x x) (λy.y)))
1,(((λx.(λy.((y y) (λz.((λa.(λb.(λc.(x (c (a c))...,((λx.((x x) (λy.((λz.(λa.(λb.((λc.(((λd.(c d))...
2,((λx.((x x) (λy.((λz.(λa.(λb.((λc.(((λd.(c d))...,((((λx.x) ((λy.y) (λz.(z (λa.z))))) ((λx.x) ((...
3,((((λx.x) ((λy.y) (λz.(z (λa.z))))) ((λx.x) ((...,((((λx.x) (λy.(y (λz.y)))) ((λa.a) ((λb.b) (λc...
4,((((λx.x) (λy.(y (λz.y)))) ((λa.a) ((λb.b) (λc...,(((λx.(x (λy.x))) ((λz.z) ((λa.a) (λb.(b (λc.b...


In [10]:
enc_tiktoken = tiktoken.encoding_for_model("gpt-3.5-turbo")
total_tokens = 0
for term in df["term"].tolist():
    total_tokens += len(enc_tiktoken.encode(term))

print(f"Total term tokens: {total_tokens}")

total_tokens = 0
for term in df["term_next_LO"].tolist():
    total_tokens += len(enc_tiktoken.encode(term))
    
print(f"Total expected tokens: {total_tokens}")

Total term tokens: 523365
Total expected tokens: 502995


# Reduce size of expected and input terms tokens

In [37]:
df_reduced = df[[len(enc_tiktoken.encode(term)) < 77 for term in df["term"].tolist()]]

In [38]:
len(df_reduced)

1019

In [39]:
total_tokens = 0
for term in df_reduced["term"].tolist():
    total_tokens += len(enc_tiktoken.encode(term))

print(f"Total term tokens: {total_tokens}")

total_tokens = 0
for term in df_reduced["term_next_LO"].tolist():
    total_tokens += len(enc_tiktoken.encode(term))
    
print(f"Total expected tokens: {total_tokens}")

Total term tokens: 52265
Total expected tokens: 44817


# Normalize terms with gpt-3.5-turbo model

In [44]:
term_answers = []

for str_term in tqdm(df_reduced["term"].tolist()):
    prompt = f"""
    Please generate the next step of reduction a lambda term. Provide only term expression.
    
    Lambda term: '''{str_term}'''
    """
    
    response = await get_completion(prompt)
    term_answers.append(response)

100%|██████████| 1019/1019 [25:18<00:00,  1.49s/it]


In [45]:
term_answers

['(λy.(λz.((λa.(((λb.(a b)) ((λc.(λd.(λe.(c (d (λj.j)))))) (λi.(λn.(λm.i))))) (z ((λi.(λn.(λm.i))) z)))))',
 '(λx.(λy.((λz.(((λa.(z a)) ((λb.(λc.(λd.(b (c (λe.e)))))) x)) x)) (y ((λj.(λi.(λn.j))) y))))',
 '(λy.(((y ((λz.(λa.(λb.z))) y)) ((λc.(λd.(λe.(c (d (λj.j)))))) (y ((λi.(λn.(λm.i))) y)))) (y ((λt.(λr.(λq.t))) y)))',
 '(λy.(((y (λz.(λa.y))) ((λb.(λc.(λd.(b (c (λe.e)))))) (y ((λj.(λi.(λn.j))) y)))) (y ((λm.(λt.(λr.m))) y)))',
 '(λx.(λy.(((y (λz.(λa.y))) (λb.(λc.((y ((λd.(λe.(λj.d))) y)) (b (λi.i)))))) (y ((λn.(λm.(λt.n))) y))))',
 '(λy.(((y (λz.(λa.y))) (λb.(λc.((y (λd.(λe.y))) (b (λj.j)))))) (y ((λi.(λn.(λm.i))) y)))',
 '(λy.(λz.(((((λa.(λb.((z (((((x z) z) (λc.(λd.(z c)))) ((y z) y)) y)) z))) (λe.z)) (λj.j)) ((z (λi.y)) (λn.y))) ((z y) z))))',
 '(λy.(λz.((((λa.((z (((((x z) z) (λb.(λc.(z b)))) ((y z) y)) y)) z)) (λd.d)) ((z (λe.y)) (λj.y))) ((z y) z))))',
 '(λx.(λy.(y (λz.((y (z (((x (λa.((y y) (((λb.y) x) (λc.(λd.x)))))) x) (λe.(λj.(j ((λi.i) y))))))) (x (λn.n))))))) (λx.(λy.(y (

In [51]:
df_reduced = df_reduced.reset_index(drop=True)

In [52]:
df_reduced["gpt3.5_answers"] = term_answers

In [54]:
df_reduced.head()

,term,term_next_LO,gpt3.5_answers
0,((λx.(λy.(λz.((λa.(((λb.(a b)) ((λc.(λd.(λe.(c...,(λx.(λy.((λz.(((λa.(z a)) ((λb.(λc.(λd.(b (c (...,(λy.(λz.((λa.(((λb.(a b)) ((λc.(λd.(λe.(c (d (...
1,(λx.(λy.((λz.(((λa.(z a)) ((λb.(λc.(λd.(b (c (...,(λx.(λy.(((λz.((y ((λa.(λb.(λc.a))) y)) z)) ((...,(λx.(λy.((λz.(((λa.(z a)) ((λb.(λc.(λd.(b (c (...
2,(λx.(λy.(((y ((λz.(λa.(λb.z))) y)) ((λc.(λd.(λ...,(λx.(λy.(((y (λz.(λa.y))) ((λb.(λc.(λd.(b (c (...,(λy.(((y ((λz.(λa.(λb.z))) y)) ((λc.(λd.(λe.(c...
3,(λx.(λy.(((y (λz.(λa.y))) ((λb.(λc.(λd.(b (c (...,(λx.(λy.(((y (λz.(λa.y))) (λb.(λc.((y ((λd.(λe...,(λy.(((y (λz.(λa.y))) ((λb.(λc.(λd.(b (c (λe.e...
4,(λx.(λy.(((y (λz.(λa.y))) (λb.(λc.((y ((λd.(λe...,(λx.(λy.(((y (λz.(λa.y))) (λb.(λc.((y (λd.(λe....,(λx.(λy.(((y (λz.(λa.y))) (λb.(λc.((y ((λd.(λe...


In [55]:
df_reduced.to_csv("./data/gpt3.5_answers.csv", index=False)

# Calculate accuracy

In [58]:
true_count = 0

for next_term, next_pred_term in zip(df_reduced["term_next_LO"], df_reduced["gpt3.5_answers"]):
    if next_term == next_pred_term:
        true_count += 1
        
print("pure accuracy: ", true_count * 100 / len(df_reduced), "%")

pure accuracy:  1.1776251226692835 %


# Calculate ROUGE and BLEU

In [60]:
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

scores = []
for ref_term, sys_term in zip(df_reduced["term_next_LO"].tolist(), df_reduced["gpt3.5_answers"].tolist()):
    score = scorer.score(ref_term, sys_term)
    scores.append(score)

In [63]:
scores

[{'rouge1': Score(precision=0.6956521739130435, recall=0.7619047619047619, fmeasure=0.7272727272727272),
  'rouge2': Score(precision=0.3181818181818182, recall=0.35, fmeasure=0.3333333333333333),
  'rougeL': Score(precision=0.4782608695652174, recall=0.5238095238095238, fmeasure=0.5)},
 {'rouge1': Score(precision=0.8095238095238095, recall=0.5862068965517241, fmeasure=0.68),
  'rouge2': Score(precision=0.25, recall=0.17857142857142858, fmeasure=0.20833333333333331),
  'rougeL': Score(precision=0.5238095238095238, recall=0.3793103448275862, fmeasure=0.44)},
 {'rouge1': Score(precision=0.8076923076923077, recall=0.84, fmeasure=0.8235294117647058),
  'rouge2': Score(precision=0.32, recall=0.3333333333333333, fmeasure=0.32653061224489793),
  'rougeL': Score(precision=0.6153846153846154, recall=0.64, fmeasure=0.6274509803921569)},
 {'rouge1': Score(precision=0.7916666666666666, recall=0.8260869565217391, fmeasure=0.8085106382978724),
  'rouge2': Score(precision=0.30434782608695654, recall=0

In [65]:
avg_scores = {
    "metric": ["ROUGE-1", "ROUGE-2", "ROUGE-L"],
    "precision": [0.0, 0.0, 0.0], 
    "recall": [0.0, 0.0, 0.0],
    "fmeasure": [0.0, 0.0, 0.0], 
}

for score in scores:
    avg_scores["precision"][0] += score["rouge1"].precision
    avg_scores["precision"][1] += score["rouge2"].precision
    avg_scores["precision"][2] += score["rougeL"].precision
    
    avg_scores["recall"][0] += score["rouge1"].recall
    avg_scores["recall"][1] += score["rouge2"].recall
    avg_scores["recall"][2] += score["rougeL"].recall
    
    avg_scores["fmeasure"][0] += score["rouge1"].fmeasure
    avg_scores["fmeasure"][1] += score["rouge2"].fmeasure
    avg_scores["fmeasure"][2] += score["rougeL"].fmeasure
    
for key_ in ("precision", "recall", "fmeasure"):
    avg_scores[key_][0] /= len(scores)
    avg_scores[key_][1] /= len(scores)
    avg_scores[key_][2] /= len(scores)
    
pd.DataFrame(avg_scores).head()

,metric,precision,recall,fmeasure
0,ROUGE-1,0.587819,0.674102,0.598760
1,ROUGE-2,0.389979,0.461713,0.402671
2,ROUGE-L,0.527615,0.605807,0.537018


In [81]:
references = [[ref_term, sys_term] for ref_term, sys_term in zip(df_reduced["term_next_LO"].tolist(), df_reduced["gpt3.5_answers"].tolist())]
candidate = df_reduced["term_next_LO"].tolist()

bleu = sacrebleu.corpus_bleu(candidate, references)
print("BLEU Score:", bleu.score)

BLEU Score: 99.84996206811196


# USE EVEN SMALLER TERMS

In [83]:
df_more_reduced = df_reduced[[len(enc_tiktoken.encode(term)) < 40 for term in df_reduced["term"].tolist()]]
df_more_reduced = df_more_reduced.reset_index(drop=True)
df_more_reduced.to_csv("./data/gpt3.5_answers_reduced.csv", index=False)

In [84]:
true_count = 0

for next_term, next_pred_term in zip(df_more_reduced["term_next_LO"], df_more_reduced["gpt3.5_answers"]):
    if next_term == next_pred_term:
        true_count += 1
        
print("pure accuracy: ", true_count * 100 / len(df_more_reduced), "%")

pure accuracy:  3.9344262295081966 %


In [87]:
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

scores = []
for ref_term, sys_term in zip(df_more_reduced["term_next_LO"].tolist(), df_more_reduced["gpt3.5_answers"].tolist()):
    score = scorer.score(ref_term, sys_term)
    scores.append(score)

In [88]:
avg_scores = {
    "metric": ["ROUGE-1", "ROUGE-2", "ROUGE-L"],
    "precision": [0.0, 0.0, 0.0], 
    "recall": [0.0, 0.0, 0.0],
    "fmeasure": [0.0, 0.0, 0.0], 
}

for score in scores:
    avg_scores["precision"][0] += score["rouge1"].precision
    avg_scores["precision"][1] += score["rouge2"].precision
    avg_scores["precision"][2] += score["rougeL"].precision
    
    avg_scores["recall"][0] += score["rouge1"].recall
    avg_scores["recall"][1] += score["rouge2"].recall
    avg_scores["recall"][2] += score["rougeL"].recall
    
    avg_scores["fmeasure"][0] += score["rouge1"].fmeasure
    avg_scores["fmeasure"][1] += score["rouge2"].fmeasure
    avg_scores["fmeasure"][2] += score["rougeL"].fmeasure
    
for key_ in ("precision", "recall", "fmeasure"):
    avg_scores[key_][0] /= len(scores)
    avg_scores[key_][1] /= len(scores)
    avg_scores[key_][2] /= len(scores)
    
pd.DataFrame(avg_scores).head()

,metric,precision,recall,fmeasure
0,ROUGE-1,0.559699,0.611534,0.565665
1,ROUGE-2,0.360110,0.409488,0.368042
2,ROUGE-L,0.523219,0.573354,0.529258


In [89]:
references = [[ref_term, sys_term] for ref_term, sys_term in zip(df_more_reduced["term_next_LO"].tolist(), df_more_reduced["gpt3.5_answers"].tolist())]
candidate = df_more_reduced["term_next_LO"].tolist()

bleu = sacrebleu.corpus_bleu(candidate, references)
print("BLEU Score:", bleu.score)

BLEU Score: 100.00000000000004


# Compare actual terms